# Molecular Dynamics simulation


**Important note:**

The animated graphs do not work with the default matplotlib backends for notebooks.  You need to set another backend, this is done with the ``%matplotlib backend`` notebook command.

* For **Windows** and **Linux**, choose the ``tk`` backend.
* For **MacOS**, choose the ``osx`` backend as it gives smoother animations on a Mac.

(the ``tk`` backend works well on Windows, but not on MacOS).

If you change the backend, you need to restart the kernel!

In [ ]:
%matplotlib osx
import numpy as np
import matplotlib.pyplot as plt

## The MD code

In [ ]:
class Atoms:
    def __init__(self, size, N, eps, sigma, mass=1.0):
        """Create a 2D atomic system.
        
        Parameters:
        size: Linear size of system
        N: Number of atoms to create.
        eps: Lennard-Jones parameter epsilon (strenght of interaction).
        sigma: Lennard-Jones parameter sigma (size of atom).
        """
        self.rng = np.random.default_rng()  # Random number generator
        self.eps = eps
        self.twelve_eps = 12 * eps
        self.sigma = sigma
        self.sigma2 = sigma * sigma
        self.mass = mass
        self.create_random_atoms(size, N)
        
    def create_random_atoms(self, size, N):
        "Create atomic system with N atoms in size*size box."
        # We should avoid placing atoms closer than sigma apart, 
        # and not right at the edge of the system.
        self.size = size  
        self.natoms = N      
        self.positions = 0.5 * self.sigma + (size - self.sigma) * self.rng.random(size=(N,2))
        # If atoms are too close, move them
        n_moved = 0
        for i in range(1, N):
            ok = False
            # Check distance to all closer atoms
            while not ok:
                vecs = self.positions[:i,:] - self.positions[i]
                d2 = vecs[:,0]**2 + vecs[:,1]**2
                ok = d2.min() > (1.4 * self.sigma)**2
                if not ok:
                    self.positions[i] = self.sigma + (size - 2*self.sigma) * self.rng.random(size=2)
                    n_moved += 1
                if n_moved > 25*N*N:
                    raise RuntimeError(f'Cannot place atoms - emergency stop!  (placed {i}/{N}).')
        if n_moved:
            print(f'Needed to move {n_moved} atoms.')

    def set_temperature(self, kT):
        """Set velocity to Maxwell-Boltzman distribution with temperature T.
        
        The temperature must be given in energy units, i.e. k_B * T.
        """
        xi = self.rng.standard_normal((self.natoms, 2))
        self.velocities = xi * np.sqrt(kT/self.mass)

    def get_ekin(self):
        "Return the total kinetic energy"
        return 0.5 * self.mass * (self.velocities**2).sum() 
    
    def get_epot(self):
        'Calculate and return potential energy.'
        # We  need to calculate interactions between all pairs of atoms.
        # Loop over all atoms, then calculate interactions with all atoms
        # that have lower index than this one.
        E = 0
        for i in range(1, self.natoms):
            r_ij_vec = self.positions[:i,:] - self.positions[i]
            sq_r_ij = (r_ij_vec * r_ij_vec).sum(axis=1)
            E += self.calc_U(sq_r_ij).sum()
        return E

    def get_forces(self):
        "Calculate and return forces on all atoms."
        # We  need to calculate interactions between all pairs of atoms.
        # Loop over all atoms, then calculate interactions with all atoms
        # that have lower index than this one.  Add force to this atom and
        # to all the others.
        F = np.zeros((self.natoms, 2))
        for i in range(1, self.natoms):
            r_ij_vec = self.positions[:i,:] - self.positions[i]
            sq_r_ij = (r_ij_vec * r_ij_vec).sum(axis=1)
            assert sq_r_ij.shape == (i,)
            # Derivative of bond energy with respect to squared interatomic distance
            dU_dr_squared = self.calc_dU_dr_sq(sq_r_ij)
            # Derivative of squared interatomic distance with respect to position
            # of this atom.  Note that a minus is taken by the minus in F = - nabla U
            F_contribution = 2 * r_ij_vec * dU_dr_squared[:,np.newaxis]
            # These are the contributions to the force on this atom.  Add to this
            # atom, and add reaction to other atoms.
            F[i] += F_contribution.sum(axis=0)
            F[:i] -= F_contribution
        return F
    
    def calc_U(self, sq_r):
        x = (self.sigma2 / sq_r)**3
        return 4 * self.eps * (x * x - x)
    
    def calc_dU_dr_sq(self, sq_r):
        x = (self.sigma2 / sq_r)**3
        return self.twelve_eps * (x - 2 * x * x) / sq_r

In [ ]:
class ReflectingBoundaries:
    'Reflecting boundary conditions.'
    def __init__(self):
        self.delta_p_accumulated = 0.0

    def apply(self, atoms):
        s = atoms.size
        # r and v are references to the positions and velocities, modifying
        # them modifies the atoms !
        r = atoms.positions
        v = atoms.velocities
        # Reflect if coordinate less than 0
        mask = r < 0.0
        r[mask] *= -1 # Move back into box
        v_old = v.copy()
        v[mask] *= -1
        self.delta_p_accumulated += (v - v_old).sum() * a.mass
        # Reflect if coordinate above size
        mask = r > s
        # r -> s - (r - s) = 2 * s - r
        r[mask] *= -1
        r[mask] += 2*s
        v_old = v.copy()
        v[mask] *= -1
        self.delta_p_accumulated += (v_old - v).sum() * a.mass

In [ ]:
class VelocityVerlet:
    'Molecular dynamics using the velocity Verlet algorithm.'
    def __init__(self, atoms, timestep):
        self.atoms = atoms
        self.dt = timestep
        self.boundary = ReflectingBoundaries()
        self.time = 0.0

    def run(self, N):
        'Run N timesteps velocity verlet'
        F = self.atoms.get_forces()
        for step in range(N):
            # Update velocities with half a timestep using initial forces
            self.atoms.velocities += (0.5 * self.dt / self.atoms.mass) * F
            # Update positions with a full timestep using intermediate velocities
            self.atoms.positions += self.dt * self.atoms.velocities
            self.boundary.apply(self.atoms)
            # Update velocities with half a timestep using final forces
            F = self.atoms.get_forces()
            self.atoms.velocities += (0.5 * self.dt / self.atoms.mass) * F
        self.time += N * self.dt

## Running the simulation

In [ ]:
initial_temperature = 0.01

size = 30  # Ångström
epsilon = 0.1  # eV
sigma = 1.0    # Å

a = Atoms(size, 20, epsilon, sigma)
a.set_temperature(initial_temperature)
#print(a.velocities)

dyn = VelocityVerlet(a, 0.004)
#print(a.get_ekin())

fig, ax = plt.subplots(1, 3, figsize=(15,5))

# Marker size: Plot size is approx 3.5 inches
markersize = 3.5 * 72 * (a.sigma / a.size)
texts = True  # Titles and legends make plots a bit slower
ax[2].set_aspect(1.0)

# Give each atom a 'trail' in the graphs, to aid seeing what happens-
# Heuristic: scale trail length with size / sqrt(T) for good-looking plots.
traillen = int(0.1 * size / np.sqrt(initial_temperature) + 1.5)
oldpos = np.zeros((traillen, a.natoms, 2))
oldpos[:] = a.positions[np.newaxis,:,:]
ekin = []
epot = []
pressure = []
avgpressure = []
for i in range(1000):
    #ax[1].plot(a.positions[:,0], a.positions[:,1], marker='o', linestyle='None', color='#f0f0f0', markersize=markersize)
    dyn.run(200)
    # Plot energies - multiply by 1000 for nicer numbers
    ekin.append(a.get_ekin() / a.natoms)
    epot.append(a.get_epot() / a.natoms)
    ek = np.array(ekin)
    ep = np.array(epot)
    ax[0].clear()
    if texts:
        ax[0].set_title('Energies [meV/atom]')
    ax[0].plot(1000*ek, label='Ekin')
    ax[0].plot(1000*ep, label='Epot')
    ax[0].plot(1000*(ek + ep), label='Etotal')
    if texts:
        ax[0].legend(loc='lower left')
    
    # Plot pressure - also multiplied by 1000
    pressure.append(dyn.boundary.delta_p_accumulated / (4 * a.size * dyn.time))
    ax[1].clear()
    if texts:
        ax[1].set_title('Pressure [meV / Å^2]')
    ax[1].plot(1000*np.array(pressure), label='p')
    # And expected pressure
    ax[1].plot(1000*(a.natoms / a.size**2) * ek, label='Ideal gas p')
    if texts:
        ax[1].legend(loc='upper left')
    # Plot atoms
    ax[2].clear()
    
    # Make the trails
    oldpos[:-1] = oldpos[1:]
    oldpos[-1] = a.positions
    for x, y in zip(oldpos[:,:,0].T, oldpos[:,:,1].T):
        ax[2].plot(x, y, color='k', markersize=0.1*markersize)
    #ax[2].plot(oldpos[:,:,0].flat, oldpos[:,:,1].flat, marker='o', linestyle='None', color='k', markersize=0.1*markersize)

    # Plot the current positions of the atoms
    ax[2].plot(a.positions[:,0], a.positions[:,1], 'bo', markersize=markersize)
    ax[2].set_xlim(0,a.size)
    ax[2].set_ylim(0,a.size)
    #oldpos[i % traillen] = a.positions
    
    #plt.show()
    plt.pause(.0001)

In [ ]:
kT = np.mean(ekin)
print("Average temperature kT =", kT)
p = pressure[-1]
delta_p = np.std(pressure)
print("Average pressure:", p, "+/-", delta_p)
expected = a.natoms / a.size**2 * kT
print("Expected pressure from ideal gas law (N kT / A):", expected)
print()
print("Multiplied by 1000 for human readability:")
print("Average pressure (x 1000):", 1000*p, "+/-", 1000*delta_p)
print("Expected pressure from ideal gas law (x 1000):", 1000*expected)


If you want to save the figure, you can run the box below.  The filename contains the size of the system, and the temperature.  The latter is multiplied by 1000 to avoid decimal points in the file name.

In [ ]:
fig.savefig(f'plot_size={size}_T={1000*initial_temperature:.0f}.png', dpi=150)

Check that the simulation time step is not too small.  For scientific simulations, variations should be less than $10^{-5}$ eV, for these exercises variations up to $10^{-2}$ are probably fine.  If you set the temperature too large, you will need to reduce the time step in ``VelocityVerlet`` to prevent energy conservation from being damaged.

In [ ]:
etot = np.array(ekin) + np.array(epot)
plt.figure()
plt.plot(etot)
print(etot.max() - etot.min())

## Radial Distribution Function

At the end of the exercise, you can try calculating RDF's.  You need to create the
object before the main loop above:
```
rfd = RadialDistributionFunction(a)
```
and then update it inside the loop (e.g. after the ``dyn.run()`` call):
```
rdf.collect_data(a)
```
Then after the simulation has run, you can plot the collected RDF data:
```
x, y = rdf.get_rdf()
plt.figure()
plt.plot(x, y)
```

To get good data, you probably need to run the loop for 1000 iterations.

In [ ]:
class RadialDistributionFunction:
    def __init__(self, atoms, nbins=100):
        self.cutoff = atoms.size / 3.0
        self.nbins = nbins
        self.ncalls = 0   # Number of times data has been collected
        self.natoms = 0   # Sum of number of atoms during collections
        self.histo = np.zeros(nbins, int)

    def collect_data(self, atoms):
        """Collect RDF data for the atoms.
        
        Only atoms in the central third of the computational box are
        considered, we can then get RDF out to a distance of 1/3 of
        the system size without hitting the boundaries.
        """
        r = atoms.positions
        # Only atoms in the central part of the box are considered
        central = ( (r[:,0] > self.cutoff) & (r[:,0] < 2*self.cutoff) &
                    (r[:,1] > self.cutoff) & (r[:,1] < 2*self.cutoff) )
        c_pos = r[central]
        # For each central atom, calculate the distances to all other atoms
        # One of them will be the atom itself, we will remove that later
        neighbor_distances = []
        for pos in c_pos:
            r_ij = r - pos
            d = np.sqrt(r_ij[:,0]**2 + r_ij[:,1]**2)
            # Keep all distances larger than 1e-9 (the atom itself)
            neighbor_distances.append(d[d > 1e-9])
        if len(neighbor_distances) == 0:
            # No data collected in this time step
            return
        neighbor_distances = np.concatenate(neighbor_distances)
        h, _ = np.histogram(neighbor_distances, bins=self.nbins, range=(0, self.cutoff))
        self.histo += h
        self.ncalls += 1
        self.natoms += len(c_pos)

    def get_rdf(self):
        """Return a normalized radial distribution function."""
        density = self.natoms / self.cutoff**2
        delta_r = self.cutoff / self.nbins
        distances = (np.arange(self.nbins) + 0.5) * delta_r
        d_area = 2 * np.pi * distances * delta_r
        normalization = 2 * density * d_area  # All pairs were counted twice
        return distances, self.histo / normalization